In [3]:
import sys
try:
    from pyathena import connect
except:
    print("Failed to import pyathena, trying to install it")
    !{sys.executable} -m pip install PyAthena
    from pyathena import connect
    print("successfully installed")
try:
    import psycopg2
except:
    print("Failed ot import psychopg2, trying to install it")
    !{sys.executable} -m pip install psycopg2-binary
    import psycopg2
    print("Successfully installed")
import pandas as pd
import boto3
import boto3
import base64
from botocore.exceptions import ClientError
from datetime  import datetime, timedelta, date
import time

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [4]:
import creds

In [5]:
# if using jupyterlab and plotly doesn't work https://plot.ly/python/getting-started/#jupyterlab-support-python-35
install_jupyterlab_plotly_extensions_command = """
# Avoid "JavaScript heap out of memory" errors during extension installation
# (OS X/Linux)
export NODE_OPTIONS=--max-old-space-size=4096
# (Windows)
set NODE_OPTIONS=--max-old-space-size=4096

# Jupyter widgets extension
jupyter labextension install @jupyter-widgets/jupyterlab-manager@1.0 --no-build

# jupyterlab renderer support
jupyter labextension install jupyterlab-plotly@1.1.0 --no-build

# FigureWidget support
jupyter labextension install plotlywidget@1.1.0 --no-build

# JupyterLab chart editor support (optional)
jupyter labextension install jupyterlab-chart-editor@1.2 --no-build

# Build extensions (must be done to activate extensions since --no-build is used above)
jupyter lab build

# Unset NODE_OPTIONS environment variable
# (OS X/Linux)
unset NODE_OPTIONS"""



    

In [6]:
import plotly.graph_objects as go
import plotly as py

import plotly.offline as off
import plotly.graph_objs as go
import numpy as np
from plotly.subplots import make_subplots

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline 


#off.init_notebook_mode(connected=True)

In [7]:
# plotly, which seems to have endless issues to get working reliably on sagemaker.
if False:
    fig = make_subplots(1, 1)

    fig.add_trace(
        go.Pie(values=[1,2,3], labels=["a", "b", "c"]),
    )


    fig.show()
    
    


In [8]:
s3_client = boto3.client('s3')
s3_resource = boto3.resource('s3') #slower, but easier
# s3_conn = boto3.s3.connect_to_region('us-west-2')

athena = boto3.client('athena')

# Settings

In [9]:
#--- Analysis Settings --
num_days_to_analyse = 4
analysis_end_time = datetime.fromordinal(date.today().toordinal()) # start of today as we only get yesterday's data.
analysis_start_time = analysis_end_time - timedelta(days=num_days_to_analyse)
country_code_to_analyse = "ID"


#--- Core Settings -- 
event_log_s3_path = "s3://ms-data-pipeline-production/ms-data-stream-production-processed"
event_log_s3_bucket = event_log_s3_path.split("s3://")[1].split("/")[0]
event_log_s3_prefix = event_log_s3_path.split("/")[-1]
athena_bucket_path = "s3://aws-athena-query-results-358002497134-ap-southeast-1/"
#athena_database = "ms_data_processed_production"
athena_database = "ms_data_lake_production"
athena_raw_events_table = "ms_data_stream_production_processed"
#athena_raw_events_table = "ms_data_stream_production_processed_cd5f4696237059a21d780afa83822e6b"
athena_year_partition = "partition_0" #2019
athena_month_partition = "partition_1" #02
athena_day_partition = "partition_2" #0
athena_easy_events_table =  "id_ab_test"
aws_region_name = "ap-southeast-1"


# --
list_out_s3_bucket = False #extremely slow.


# Getting Base Data

## Events from Athena

In [10]:
print("Connecting to Athena")
athena_conn = connect(s3_staging_dir=athena_bucket_path, region_name=aws_region_name)

Connecting to Athena


In [11]:
print("Running small test query to check the athena connection")
pd.read_sql("select * from "+athena_database+"."+athena_raw_events_table+" limit 10", athena_conn)

Running small test query to check the athena connection


,type,context,user,body,sent_at,partition_0,partition_1,partition_2
0,page,{title=Cara Membuat Kartu Keluarga Tanpa Syara...,"{anonymous_id=null, amp_id=avNftn1pk8WETSiBkGC...","{event_name=PageView, data={title=null, status...",2019-10-19T11:41:05.317000+00:00,2019,10,19
1,event,"{title=Besaran Bunga OVO Paylater, Traveloka, ...","{anonymous_id=null, amp_id=amp-gtzEl-tj_-yvn4Z...","{event_name=Reading, data={title=null, status=...",2019-10-19T11:41:05.655000+00:00,2019,10,19
2,page,{title=【2019酒店自助餐優惠折扣】Buffet買一送一 低至$233！ | Mon...,{anonymous_id=6ba9edde-e271-47d2-9a43-ec4b72b1...,"{event_name=PageView, data={title=null, status...",2019-10-19T11:41:04.617Z,2019,10,19
3,event,{title=BNI Sekuritas: Ini Lho Keuntungannya da...,"{anonymous_id=null, amp_id=V4LZ8ePrav9013BwKqd...","{event_name=Reading, data={title=null, status=...",2019-10-19T11:41:08.057000+00:00,2019,10,19
4,page,{title=Begini Cek Saldo Taspen Online yang Res...,{anonymous_id=3e819cc7-67aa-4dce-9f78-dcae0e83...,"{event_name=PageView, data={title=null, status...",2019-10-19T11:41:02.764Z,2019,10,19
5,page,{title=Salut! 5 Artis Indonesia Ini Gak Malu M...,{anonymous_id=3debdbd9-27a7-4963-a268-956e99cf...,"{event_name=PageView, data={title=null, status...",2019-10-19T11:40:53.251Z,2019,10,19
6,event,"{title=Kekayaan Raffi Ahmad Vs Ruben Onsu, Man...",{anonymous_id=f053d693-baff-4894-a8ba-9260f2af...,"{event_name=Reading, data={title=Kekayaan Raff...",2019-10-19T11:41:03.855Z,2019,10,19
7,event,{title=Keluarga Halilintar Kantongi Penghasila...,{anonymous_id=0b6ce28e-2439-4ac6-9966-a11b80b1...,"{event_name=Reading, data={title=Keluarga Hali...",2019-10-19T11:41:05.494Z,2019,10,19
8,event,"{title=AHY Huni Rumah Dinas TNI Pertama, Begin...",{anonymous_id=a1fe77c2-414f-4766-a4f1-c1f58c1d...,"{event_name=Reading, data={title=AHY Huni Ruma...",2019-10-19T11:41:06.351Z,2019,10,19
9,event,{title=Car Loan in Singapore - Guide to Financ...,{anonymous_id=133e7dea-4ef9-4eba-bf5c-1cc4168e...,"{event_name=Reading, data={title=Car Loan in S...",2019-10-19T11:41:06.653Z,2019,10,19


In [12]:
def create_blog_pageview_and_scroll_query(country_code, from_datetime, to_datetime, limit=None, include_device_type_data = True):
    """ 
    This has been taken from the view in athena to make it more flexible.
    country_code should be e.g. TW / SG / ...
    
    It filters out just events from the blog
    """
    print("Getting the pageview and read events from %s to %s"%(from_datetime.isoformat(), to_datetime.isoformat()))
    
    country_code_to_url_core = {
        "ID":"moneysmart.id", 
        "PH":"moneysmart.ph", 
        "TW":"moneysmart.tw", 
        "SG":"blog.moneysmart.sg", 
        "HK":"blog.moneysmart.hk"
        
    }
    
    if country_code not in country_code_to_url_core:
        raise Exception("Didn't recognise country code"+repr(country_code)+".")
        
    #These are just used for the partition selection and further filter is done on the datetime level.
    start_year_inclusive = from_datetime.year
    end_year_inclusive = to_datetime.year
    start_month_inclusive = from_datetime.month
    end_month_inclusive = to_datetime.month
    start_day_inclusive = from_datetime.day
    end_day_inclusive = to_datetime.day
        
    query = """
    SELECT
      CAST("from_iso8601_timestamp"("sent_at") AS timestamp) "sent_at_timestamp"
    , "sent_at"
    , "type"
    , "body"."event_name"
    , "body"."data"."status"
    , (CASE "body"."data"."status" 
        WHEN 'Page Bottom Reached' THEN 100 
        WHEN 'Article Reading Completed' THEN 100 WHEN 'Article Body 100' THEN 100 
        WHEN 'Article Reading 75%' THEN 75 WHEN 'Article Body 75' THEN 75 
        WHEN 'Article Reading 50%' THEN 50 WHEN 'Article Body 50' THEN 50 
        WHEN 'Article Reading 25%' THEN 25 WHEN 'Article Body 25' THEN 25
        WHEN 'Article Reading Started' THEN 0 WHEN 'Article Loaded' THEN 0 
        ELSE 0 END) "article_read_depth"
    , "user"."anonymous_id"
    , "split_part"("context"."page_url", '?', 1) "canonical_url"
    , "context"."page_url"
    , "context"."referrer"
    , CAST("strpos"("context"."page_url", '?amp') AS boolean) "is_amp"
    , CAST((("context"."page_url" LIKE '%utm_medium%') OR ("context"."page_url" LIKE '%gclid%')) AS boolean) "has_marketing_param"
    , CAST("strpos"("context"."page_url", '://www3.') AS boolean) "is_test"
    , CAST("strpos"("context"."page_url", '://www.') AS boolean) "is_control"
    """
    if include_device_type_data:
        query += """
        , context.device.type as device_type
        , context.browser.name as browser_name
        , context.browser.major as browser_major_version
        , context.browser.version as browser_version
        
        , context.operating_system.name as operating_system_name
        , context.operating_system.version as operating_system_version
        , context.user_agent as user_agent
        """
        
    query += """
    FROM
      """+athena_database+ "." +athena_raw_events_table+ """
    WHERE 
        (("body"."event_name" = 'Reading') OR ("body"."event_name" = 'PageView')) 
        
"""
    
    country_constraint = """ AND 
            ("regexp_extract"("context"."page_url", '^(([^:/?#]+):)?(//([^/?#]*))?([^?#]*)(\?([^#]*))?(#(.*))?', 4) 
            LIKE '%{}%')""".format(country_code_to_url_core[country_code])

    
    partition_constraints = "\n AND ("
    partition_constraints += "\n " + athena_year_partition + " >= '%.4i'"%start_year_inclusive
    partition_constraints += "\n AND " + athena_month_partition + " >= '%.2i'"%start_month_inclusive
    partition_constraints += "\n AND " + athena_day_partition+ " >= '%.2i'"%start_day_inclusive
    
    #TOD: >> I think this needs some adjustment for when you roll between months.
    partition_constraints += "\n AND " + athena_year_partition + " <= '%.4i'"%end_year_inclusive
    partition_constraints += "\n AND " + athena_month_partition + " <= '%.2i'"%end_month_inclusive
    partition_constraints += "\n AND " + athena_day_partition+ " <= '%.2i'"%end_day_inclusive
    partition_constraints += "\n) "
     # TODO: + datetime further constraints
    
    query = query + " "+country_constraint
    query = query+ " " + partition_constraints
    
    if limit:
        query += " LIMIT %i"%limit
    
    return query
    
def get_blog_events(country_code, from_datetime, to_datetime, limit=None):
    #TODO: ideally you break into chunks during reading and replace various strings with post_id and set data types to more efficient ones (category)
    start_time =  datetime.now()
    print("Starting athena events query at %s"% start_time.isoformat())
    print("This might take some time")
    
    query = create_blog_pageview_and_scroll_query(country_code, from_datetime, to_datetime, limit=limit)
    
    print("Query : "+ query)
    
    events = pd.read_sql(query, athena_conn)
    
    end_time =  datetime.now()
    print("Ended query at %s"%end_time.isoformat())
    dt = end_time-start_time
    print("Query took %s" %dt)
    return events
    

In [13]:
events = get_blog_events(country_code_to_analyse, analysis_start_time, analysis_end_time, limit =None)


Starting athena events query at 2019-11-05T07:18:18.482265
This might take some time
Getting the pageview and read events from 2019-11-01T00:00:00 to 2019-11-05T00:00:00
Query : 
    SELECT
      CAST("from_iso8601_timestamp"("sent_at") AS timestamp) "sent_at_timestamp"
    , "sent_at"
    , "type"
    , "body"."event_name"
    , "body"."data"."status"
    , (CASE "body"."data"."status" 
        WHEN 'Page Bottom Reached' THEN 100 
        WHEN 'Article Reading Completed' THEN 100 WHEN 'Article Body 100' THEN 100 
        WHEN 'Article Reading 75%' THEN 75 WHEN 'Article Body 75' THEN 75 
        WHEN 'Article Reading 50%' THEN 50 WHEN 'Article Body 50' THEN 50 
        WHEN 'Article Reading 25%' THEN 25 WHEN 'Article Body 25' THEN 25
        WHEN 'Article Reading Started' THEN 0 WHEN 'Article Loaded' THEN 0 
        ELSE 0 END) "article_read_depth"
    , "user"."anonymous_id"
    , "split_part"("context"."page_url", '?', 1) "canonical_url"
    , "context"."page_url"
    , "context"."re

In [55]:
print("Got %i events"%len(events))
events.head(5)

Got 2833515 events


,sent_at_timestamp,sent_at,type,event_name,status,article_read_depth,anonymous_id,canonical_url,page_url,referrer,...,has_marketing_param,is_test,is_control,device_type,browser_name,browser_major_version,browser_version,operating_system_name,operating_system_version,user_agent
0,2019-11-02 10:22:04.201,2019-11-02T10:22:04.201Z,event,Reading,Article Body 100,100,d579da83-993e-4961-8055-706bd379332d,https://www.moneysmart.id/rumah-mewah-bebby-fey/,https://www.moneysmart.id/rumah-mewah-bebby-fe...,http://m.facebook.com/,...,True,False,True,mobile,Facebook,245,245.0.0.39.118,Android,9,Mozilla/5.0 (Linux; Android 9; ASUS_X00TD Buil...
1,2019-11-02 10:22:15.320,2019-11-02T10:22:15.320Z,event,Reading,Article Body 75,75,5382bf0c-e3a0-439e-ba56-7a978fffca4c,https://www.moneysmart.id/cicilan-tanpa-kartu-...,https://www.moneysmart.id/cicilan-tanpa-kartu-...,,...,False,False,True,mobile,Samsung Browser,4,4.2,Android,6.0.1,Mozilla/5.0 (Linux; Android 6.0.1; SAMSUNG SM-...
2,2019-11-02 10:22:04.209,2019-11-02T10:22:04.209Z,event,Reading,Article Body 100,100,a3e785d5-5c74-49fc-aec5-9abee105a5d2,https://www.moneysmart.id/asisten-artis-jadi-s...,https://www3.moneysmart.id/asisten-artis-jadi-...,https://l.facebook.com/,...,True,True,False,mobile,Chrome,75,75.0.3770.101,Android,7.1.2,Mozilla/5.0 (Linux; Android 7.1.2; Redmi 5A) A...
3,2019-11-02 10:22:01.281,2019-11-02T10:22:01.281Z,event,Reading,Article Body 50,50,8b810256-8e24-414e-9755-14c16391ba28,https://www.moneysmart.id/maudy-ayunda-dan-keh...,https://www.moneysmart.id/maudy-ayunda-dan-keh...,http://m.facebook.com/,...,True,False,True,mobile,Facebook,242,242.0.0.43.119,Android,9,Mozilla/5.0 (Linux; Android 9; SM-J610F Build/...
4,2019-11-02 10:22:02.835,2019-11-02T10:22:02.835Z,event,Reading,Article Body 50,50,cdb1adb9-ccfb-403e-8437-4d64d774a826,https://www.moneysmart.id/presiden-ri-dan-peso...,https://www.moneysmart.id/presiden-ri-dan-peso...,https://l.facebook.com/,...,True,False,True,mobile,Chrome,78,78.0.3904.62,Android,5.0.2,Mozilla/5.0 (Linux; Android 5.0.2; Mi 4i) Appl...


In [59]:
#TODO: FIX PROPERLY

# hacky fix for canonical not being right in original query (and if doing this way, there's probably a quicker option)
for i in range(len(events)):
    #print(events.at[i,'canonical_url'])
    events.at[i,'canonical_url'] = events.at[i,'canonical_url'].replace("https://www3", "https://www")
    # print(events.at[i,'canonical_url'])
    #if i>100: break

In [28]:
# pageview_events = events[events.event_name=="PageView"]

In [ ]:
#pageview_events.head()

In [64]:
events[events.canonical_url.str.contains("www3")].head()

,sent_at_timestamp,sent_at,type,event_name,status,article_read_depth,anonymous_id,canonical_url,page_url,referrer,...,has_marketing_param,is_test,is_control,device_type,browser_name,browser_major_version,browser_version,operating_system_name,operating_system_version,user_agent


In [60]:
events.head()

,sent_at_timestamp,sent_at,type,event_name,status,article_read_depth,anonymous_id,canonical_url,page_url,referrer,...,has_marketing_param,is_test,is_control,device_type,browser_name,browser_major_version,browser_version,operating_system_name,operating_system_version,user_agent
0,2019-11-02 10:22:04.201,2019-11-02T10:22:04.201Z,event,Reading,Article Body 100,100,d579da83-993e-4961-8055-706bd379332d,https://www.moneysmart.id/rumah-mewah-bebby-fey/,https://www.moneysmart.id/rumah-mewah-bebby-fe...,http://m.facebook.com/,...,True,False,True,mobile,Facebook,245,245.0.0.39.118,Android,9,Mozilla/5.0 (Linux; Android 9; ASUS_X00TD Buil...
1,2019-11-02 10:22:15.320,2019-11-02T10:22:15.320Z,event,Reading,Article Body 75,75,5382bf0c-e3a0-439e-ba56-7a978fffca4c,https://www.moneysmart.id/cicilan-tanpa-kartu-...,https://www.moneysmart.id/cicilan-tanpa-kartu-...,,...,False,False,True,mobile,Samsung Browser,4,4.2,Android,6.0.1,Mozilla/5.0 (Linux; Android 6.0.1; SAMSUNG SM-...
2,2019-11-02 10:22:04.209,2019-11-02T10:22:04.209Z,event,Reading,Article Body 100,100,a3e785d5-5c74-49fc-aec5-9abee105a5d2,https://www.moneysmart.id/asisten-artis-jadi-s...,https://www3.moneysmart.id/asisten-artis-jadi-...,https://l.facebook.com/,...,True,True,False,mobile,Chrome,75,75.0.3770.101,Android,7.1.2,Mozilla/5.0 (Linux; Android 7.1.2; Redmi 5A) A...
3,2019-11-02 10:22:01.281,2019-11-02T10:22:01.281Z,event,Reading,Article Body 50,50,8b810256-8e24-414e-9755-14c16391ba28,https://www.moneysmart.id/maudy-ayunda-dan-keh...,https://www.moneysmart.id/maudy-ayunda-dan-keh...,http://m.facebook.com/,...,True,False,True,mobile,Facebook,242,242.0.0.43.119,Android,9,Mozilla/5.0 (Linux; Android 9; SM-J610F Build/...
4,2019-11-02 10:22:02.835,2019-11-02T10:22:02.835Z,event,Reading,Article Body 50,50,cdb1adb9-ccfb-403e-8437-4d64d774a826,https://www.moneysmart.id/presiden-ri-dan-peso...,https://www.moneysmart.id/presiden-ri-dan-peso...,https://l.facebook.com/,...,True,False,True,mobile,Chrome,78,78.0.3904.62,Android,5.0.2,Mozilla/5.0 (Linux; Android 5.0.2; Mi 4i) Appl...


## Convert Raw Pageview Events Into Scroll Depth Summary

Basically re-creating:

```
CREATE OR REPLACE VIEW id_pageview_metrics AS 
SELECT
  "anonymous_id"
, "canonical_url"
, "substr"("sent_at", 1, 10) "day_str"
, "min"("sent_at_timestamp") "from_time"
, "max"("sent_at_timestamp") "to_time"
, "date_diff"('second', "min"("sent_at_timestamp"), "max"("sent_at_timestamp")) "time_on_page_seconds"
, "max"("is_test") "is_test"
, "max"("article_read_depth") "max_article_read_depth"
FROM
  id_ab_test
WHERE (NOT "is_amp")
GROUP BY 1, 2, 3   >>> should it not have is_test in the group by????
```

In [113]:
# Group by page and user to within the same day to get an idea of engagement within the page 
# (if there's only one event on the page, then it's likely there was no 2nd pageview, not perfect though.)

# TODO: add is_amp

engagement_summary = events.groupby(["anonymous_id", "canonical_url", events.sent_at_timestamp.dt.floor('d'), "is_test", "is_amp"])\
    .aggregate({"sent_at_timestamp": [np.min, np.max], "article_read_depth":np.max }, axis="columns")

engagement_summary["time_on_page_seconds"] = engagement_summary.apply(lambda row: row[] - row[], axis=1)

#>>> TODO: add the time on the page as time_on_page_seconds, which I think you could do as appending a column

#non_amp_engagement_summary = engagement_summary[engagement_summary.is_amp==False]

TypeError: unsupported operand type(s) for -: 'function' and 'function'

In [118]:
engagement_summary.reset_index(inplace=True)
engagement_summary.head()


anonymous_id  \
                                          
0  00005691-02d9-48c2-ba12-b8939ccbce34   
1  0000b293-f267-410d-85fc-5895dc68aa3b   
2  0000cc5d-4dd3-46ac-80e0-567192a868fe   
3  0000e1aa-7786-4276-b928-367a16e0901b   
4  00010b6a-aa66-4d61-8817-e4963e43a24f   

                                       canonical_url sent_at_timestamp  \
                                                                         
0  https://www.moneysmart.id/utang-luar-negeri-in...        2019-09-20   
1  https://www.moneysmart.id/rumah-luwih-hotel-ba...        2019-09-20   
2  https://www.moneysmart.id/gak-sesulit-yang-dib...        2019-09-20   
3  https://www.moneysmart.id/agnez-mo-pakai-baran...        2019-09-20   
4  https://www.moneysmart.id/johan-budi-memiliki-...        2019-09-20   

  is_test is_amp       sent_at_timestamp                          \
                                    amin                    amax   
0   False  False 2019-09-20 10:00:45.039 2019-09-20 10:01:20.870   
1   False  False 2019-09-20 00:50:23.083 2019-09-20 00:52:23.595   
2   False  False 2019-09-20 08:26:14.978 2019-09-20 08:27:21.581   
3   False  False 2019-09-20 12:52:48.356 2019-09-20 12:53:17.308   
4   False  False 2019-09-20 14:15:56.913 2019-09-20 14:18:05.452   

  article_read_depth  
                amax  
0                 50  
1                100  
2                 75  
3                100  
4                100

# Queries to Add
```

select * from id_pageview_metrics where time_on_page_seconds <5 and max_article_read_depth = 100 limit 100
```
---
```
select * from id_engagement_summary order by day_str, is_test 

```

 BVHSjgnqHpqPRgHHaG9sYdJqD94ZeykJ


In [119]:
pps_query = """
select users.day_str, users.is_test, num_users, num_pageviews, cast(num_pageviews as DOUBLE) / (num_users) as pageviews_per_user_per_day from 

(select day_str, is_test, count(distinct(anonymous_id)) AS num_users from id_pageview_metrics group by day_str, is_test) as users

JOIN

(select day_str, is_test, count(*) as num_pageviews from id_pageview_metrics group by day_str, is_test) as pageviews

ON 
 users.day_str = pageviews.day_str
 and 
 users.is_test = pageviews.is_test
order by day_str, is_test
"""
pageviews_per_session = pd.read_sql_query(pps_query, athena_conn)

In [84]:
pageviews_per_session


,day_str,is_test,num_users,num_pageviews,pageviews_per_user_per_day
0,2019-08-29,False,22667,27063,1.193938
1,2019-08-29,True,778,1003,1.289203
2,2019-08-30,False,147652,178157,1.206601
3,2019-08-30,True,5432,6586,1.212445
4,2019-08-31,False,146489,177051,1.208630
5,2019-08-31,True,4999,5962,1.192639
6,2019-09-01,False,150252,180434,1.200876
7,2019-09-01,True,5429,6314,1.163013
8,2019-09-02,False,150894,182569,1.209916
9,2019-09-02,True,5893,7009,1.189377


In [85]:
pageviews_per_session.groupby(['is_test']).mean()

,num_users,num_pageviews,pageviews_per_user_per_day
is_test,,,
False,130656.857143,157664.428571,1.205112
True,4778.571429,5664.714286,1.198417


In [308]:
post_slugs_to_id_df.head()

,post_id
0,153.0
1,155.0
2,158.0
3,161.0
4,199.0


### Scalyr Setup

Functions to use later.  See https://www.scalyr.com/help/api#query

In [ ]:
def query_scalyr():
    api_key = creds.scalyr_api_key
    query_url = https://www.scalyr.com/api/query
    log_file_query_id_prod_pwa = "log=*fe-pwa-id-production*"
    log_file_query_id_pod_wordpress = 
    ok_statuses = list(range(200,210))+ list(range(300, 310))
    meh_statuses = [404,]
    bad_statuses = list(range(500, 510))
    print("Bad statuses", bad_statuses)
    
"""
{
  "token":             "xxx",
  "queryType":         "log",
  "filter":            "...",
  "startTime":         "...",
  "endTime":           "...",
  "maxCount":          nnn,
  "pageMode":          "...",
  "columns":           "...",
  "continuationToken": "...",
  "priority":          "..."
}
"""

"""
{
  "status": "success",
  "matches": [
    {
      "timestamp": "1393009097459537089",
      "message": "hello, world",
      "severity": 3,
      "session": "sess_d5952fdd-eed2-45f1-8106-b2f2af55dabd",
      "thread": "28",
      "fields": {
        "tag": "foo"
      }
    }
  ],
  "sessions": {
    "sess_d5952fdd-eed2-45f1-8106-b2f2af55dabd": {
      "serverHost": "some.host.name",
      "serverIP": "1.2.3.4",
      "session": "sess_d5952fdd-eed2-45f1-8106-b2f2af55dabd"
    }
  },
  "executionTime": 12,
  "continuationToken": "L6_ckjI4BgATVPZkxGaUwQ--"
}

"""


# Data Cleaning

* Remove what looks like bots / any really heavy users (not sure if most bots will trigger analytics events)


In [13]:
non_amp_events = events[events.is_amp==False]

# Sense Checks

## Overall AB Split

Theories about why the AB split won't match what is "set" on the AB splitter:

* Repeat users will almost always be in the control set, and we have built up a good backlog over a couple of months (40% of our users are returning from google analytics, but that is over full lifetime of the cookie https://analytics.google.com/analytics/web/?authuser=2#/report/visitors-type/a117250120w173910722p173182279/ )
* Potenially it's related to coming from AMP (don't think so)
* Potentially it's based on origin of the traffic (e.g. certain query params, but this has been turned off).

In [3]:
ab_split_summary = non_amp_events.groupby("is_test").count()["sent_at"]
ab_split_summary_pretty = [[("Test (v2)" if z[0] else "Control (v1)"), z[1]] for z in ab_split_summary.iteritems()]


import plotly.graph_objects as go
fig = make_subplots(1, 1)

fig.add_trace(
    go.Pie(values=[z[1] for z in ab_split_summary_pretty], labels=[z[0] for z in ab_split_summary_pretty]),
)

                
fig.show()

NameError: name 'non_amp_events' is not defined

## Type of Event

In [32]:
event_name_counts = events.groupby(events.event_name)
event_name_counts


In [33]:
# Doing this sense check on the event fields, not the computed depth
expected_statuses = ["Article Body {}".format(i) for i in [0, 25, 50, 75, 100]]
expected_statuses_set = set(expected_statuses)
reading_status_counts = events[events.event_name=="Reading"].groupby([events.status, events.is_test]).count()['sent_at'] #[["event_name", "is_test" ], :]
reading_status_counts.sort_index()
reading_status_counts = [[z[0][0], z[0][1], z[1]] for z in reading_status_counts.iteritems()]
test_reading_status_counts = [z for z in reading_status_counts if z[1]==True]
control_reading_status_counts = [z for z in reading_status_counts if z[1]==True]

for name, reading_statuses in [["Test", test_reading_status_counts], ["Control", control_reading_status_counts]]:
    statuses_set = set([z[0] for z in reading_statuses])
    missing_statuses = list(expected_statuses_set - statuses_set)
    extra_statuses = list(statuses_set - expected_statuses_set)
    
    if len(missing_statuses)>0:
        print("ERROR: Missing expected statuses for Reading event in "+name+": " + ", ".join(missing_statuses))
        
    if len(extra_statuses)>0:
        print("ERROR: Unexpected statuses for Reading event in "+name+": " + ", ".join(extra_statuses))
                        
# Going to look at scroll depth comparison lower down, so don't need to worry about it just now.


ERROR: Missing expected statuses for Reading event in Test: Article Body 0
ERROR: Missing expected statuses for Reading event in Control: Article Body 0


## Pageviews Grouped

* By category
* By device
* By query params (limited to the top n, and grouping glcid and fbclid)

... and for each one give the v2 percentage (which might need another chart)

## Distribution Over Time

In [262]:
# Group by hour and plot with time axis
events.groupby([events.sent_at_timestamp.hour, events.is_test]).count()

AttributeError: 'Series' object has no attribute 'hour'

# Scroll Depth Distributions

By split:
* How far down the page does a user scroll on average (table and plot)
* Time to reach a point down the page (charts) 


# Pageview Per Session

* Where they click (future)
* 

# Exploring by Browser / Device

We've seen issues on Babe, UCBrowser etc.  Need to check if there are issues from those platforms.

The issues we saw likely resulted in scroll depth issues.  I'm also not sure what analytics we're going to get from those platform, so just generally need to poke it.

## Hacky Events By Device Type

In [34]:
events.head()
events[events.event_name=="PageView"].head()

,sent_at_timestamp,sent_at,type,event_name,status,article_read_depth,anonymous_id,canonical_url,page_url,referrer,...,has_marketing_param,is_test,is_control,device_type,browser_name,browser_major_version,browser_version,operating_system_name,operating_system_version,user_agent
2,2019-11-02 22:58:59.894,2019-11-02T22:58:59.894Z,page,PageView,None,0,47207122-b1f2-440e-a739-f7fae393cf45,https://www.moneysmart.id/eross-sheila-on-7-hi...,https://www.moneysmart.id/eross-sheila-on-7-hi...,,...,True,False,True,mobile,None,None,None,Android,27,BacaBerita App/11.1.3 (Linux; U; Android 27; e...
4,2019-11-02 22:59:01.179,2019-11-02T22:59:01.179Z,page,PageView,None,0,9a6d9a78-4194-433c-9836-236304acf87f,https://www.moneysmart.id/tunjangan-naik-70-pe...,https://www.moneysmart.id/tunjangan-naik-70-pe...,https://www.google.co.id/,...,False,False,True,mobile,UCBrowser,12,12.13.2.1208,Android,7.0,Mozilla/5.0 (Linux; U; Android 7.0; en-US; viv...
7,2019-11-02 22:59:03.056,2019-11-02T22:59:03.056Z,page,PageView,None,0,17cc246a-09cf-460a-9039-d463ee955af4,https://www.moneysmart.id/zainut-tauhid-wakil-...,https://www.moneysmart.id/zainut-tauhid-wakil-...,https://api.dable.io/widgets/id/2XnLEJ7d/users...,...,False,False,True,mobile,Facebook,245,245.0.0.39.118,Android,8.1.0,Mozilla/5.0 (Linux; Android 8.1.0; CPH1823 Bui...
9,2019-11-02 22:59:01.856,2019-11-02T22:59:01.856Z,page,PageView,None,0,8f57ca70-af74-4395-9433-fcad2a536352,https://www.moneysmart.id/cara-umi-pipik-hasil...,https://www.moneysmart.id/cara-umi-pipik-hasil...,,...,True,False,True,mobile,None,None,None,Android,25,BacaBerita App/12.2.2 (Linux; U; Android 25; e...
12,2019-11-02 22:59:03.188,2019-11-02T22:59:03.188Z,page,PageView,None,0,b8d6ebc8-4acf-491c-b7ac-d2e32d51bc5a,https://www3.moneysmart.id/penghasilan-bill-ga...,https://www3.moneysmart.id/penghasilan-bill-ga...,https://www-moneysmart-id.cdn.ampproject.org/v...,...,False,True,False,mobile,Chrome,78,78.0.3904.62,Android,6.0.1,Mozilla/5.0 (Linux; Android 6.0.1; Redmi 3S) A...


In [21]:
# Get the events for a given day by device type, across all URLs


""", context.device.type as device_type
        , context.browser.name as browser_name
        , context.browser.major as browser_major_version
        , context.browser.version as browser_version
        
        , context.operating_system.name as operating_system_name
        , context.operating_system.version as operating_system_version
        , context.user_agent as user_agent"""

#cols_to_group_by = ["event_name", "status",  "device_type", "browser_name", "browser_version"]
cols_to_group_by = ["event_name", "status",  "device_type", "browser_name"]
# cols_to_group_by = ["event_name", "status",  "device_type", "browser_name", "user_agent"]
#cols_to_group_by  = ["event_name", "status",  "device_type", ]
# combined_raw_df  = events.groupby(cols_to_group_by+["is_test",]).size()

combined_raw_df = pd.DataFrame({"count":events.fillna(0).groupby(cols_to_group_by+["is_test",]).size()}).reset_index()  #fillna(-1) is to stop it removing Nulls 

# combined_raw_df[combined_raw_df.event_name=="PageView"]


In [22]:
# combined_raw_df[combined_raw_df.is_test==True]

In [23]:


# join across both sides of the AB test
test_df =combined_raw_df[combined_raw_df.is_test==True]
test_df = test_df.rename(columns={"count":"test_count"}) #in_place = True
control_df = combined_raw_df[combined_raw_df.is_test==False]
control_df = control_df.rename(columns={"count":"control_count"})

print(test_df.head(20))

#print(control_df.head())



#print(test_df.keys)
test_df.reset_index()
#print(20*"-")
#print(test_df.keys())
#print(test_df.columns)
test_df
control_df.reset_index()
test_df = test_df.set_index(cols_to_group_by)
control_df = control_df.set_index(cols_to_group_by)

combined_df = test_df.join(control_df, rsuffix="_").reset_index()
#combined_df.fillna(0) #get rid of NaN

   event_name status device_type     browser_name  is_test  test_count
3    PageView      0     desktop                0     True          13
5    PageView      0     desktop  Android Browser     True           7
7    PageView      0     desktop           Chrome     True        5486
9    PageView      0     desktop  Chrome Headless     True         121
11   PageView      0     desktop   Chrome WebView     True          13
13   PageView      0     desktop         Chromium     True          23
15   PageView      0     desktop             Edge     True         103
18   PageView      0     desktop         Facebook     True          10
20   PageView      0     desktop          Firefox     True        1780
22   PageView      0     desktop               IE     True          54
24   PageView      0     desktop         IEMobile     True           1
27   PageView      0     desktop     MIUI Browser     True           2
29   PageView      0     desktop          Maxthon     True           5
33   P

In [24]:
combined_df["test_fraction"] = combined_df.apply (lambda row: row.test_count/(row.control_count + row.test_count), axis=1)

In [39]:
#combined_df[combined_df['control_count']>1000][combined_df['browser_name']=="Facebook"].sort_values(["browser_name", "browser_version", "status"]) #.to_csv("results.csv")
combined_df[combined_df['control_count']>1000][(combined_df['event_name']=="PageView")  | (combined_df['status']=="Article Body 100")].sort_values(["device_type","browser_name","status"]) #.to_csv("results.csv")
# "test_fraction", 

# df = combined_df[combined_df['control_count']>100].sort_values(["user_agent", "event_name","status"])
# df.to_csv("test_fraction_by_user_agent.csv")

# combined_df[combined_df['control_count']>1000].sort_values(["browser_name", "browser_version", "status"]) #.to_csv("results.csv")

#combined_df[combined_df['control_count']>100][combined_df['test_count']>50].sort_values(["test_fraction"])


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:2: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



,event_name,status,device_type,browser_name,is_test,test_count,is_test_,control_count,test_fraction
2,PageView,0,desktop,Chrome,True,5486,False,56438,0.088592
55,Reading,Article Body 100,desktop,Chrome,True,3106,False,31575,0.089559
3,PageView,0,desktop,Chrome Headless,True,121,False,1188,0.092437
8,PageView,0,desktop,Firefox,True,1780,False,16450,0.097641
60,Reading,Article Body 100,desktop,Firefox,True,1055,False,9670,0.098368
13,PageView,0,desktop,Opera,True,118,False,1291,0.083747
16,PageView,0,desktop,Safari,True,173,False,2118,0.075513
66,Reading,Article Body 100,desktop,Safari,True,92,False,1180,0.072327
22,PageView,0,mobile,0,True,13235,False,124210,0.096293
23,PageView,0,mobile,Android Browser,True,1128,False,8570,0.116313


## Hacky Check That No Pages Have Issues

In [40]:
events.head()


,sent_at_timestamp,sent_at,type,event_name,status,article_read_depth,anonymous_id,canonical_url,page_url,referrer,...,has_marketing_param,is_test,is_control,device_type,browser_name,browser_major_version,browser_version,operating_system_name,operating_system_version,user_agent
0,2019-11-02 10:22:04.201,2019-11-02T10:22:04.201Z,event,Reading,Article Body 100,100,d579da83-993e-4961-8055-706bd379332d,https://www.moneysmart.id/rumah-mewah-bebby-fey/,https://www.moneysmart.id/rumah-mewah-bebby-fe...,http://m.facebook.com/,...,True,False,True,mobile,Facebook,245,245.0.0.39.118,Android,9,Mozilla/5.0 (Linux; Android 9; ASUS_X00TD Buil...
1,2019-11-02 10:22:15.320,2019-11-02T10:22:15.320Z,event,Reading,Article Body 75,75,5382bf0c-e3a0-439e-ba56-7a978fffca4c,https://www.moneysmart.id/cicilan-tanpa-kartu-...,https://www.moneysmart.id/cicilan-tanpa-kartu-...,,...,False,False,True,mobile,Samsung Browser,4,4.2,Android,6.0.1,Mozilla/5.0 (Linux; Android 6.0.1; SAMSUNG SM-...
2,2019-11-02 10:22:04.209,2019-11-02T10:22:04.209Z,event,Reading,Article Body 100,100,a3e785d5-5c74-49fc-aec5-9abee105a5d2,https://www3.moneysmart.id/asisten-artis-jadi-...,https://www3.moneysmart.id/asisten-artis-jadi-...,https://l.facebook.com/,...,True,True,False,mobile,Chrome,75,75.0.3770.101,Android,7.1.2,Mozilla/5.0 (Linux; Android 7.1.2; Redmi 5A) A...
3,2019-11-02 10:22:01.281,2019-11-02T10:22:01.281Z,event,Reading,Article Body 50,50,8b810256-8e24-414e-9755-14c16391ba28,https://www.moneysmart.id/maudy-ayunda-dan-keh...,https://www.moneysmart.id/maudy-ayunda-dan-keh...,http://m.facebook.com/,...,True,False,True,mobile,Facebook,242,242.0.0.43.119,Android,9,Mozilla/5.0 (Linux; Android 9; SM-J610F Build/...
4,2019-11-02 10:22:02.835,2019-11-02T10:22:02.835Z,event,Reading,Article Body 50,50,cdb1adb9-ccfb-403e-8437-4d64d774a826,https://www.moneysmart.id/presiden-ri-dan-peso...,https://www.moneysmart.id/presiden-ri-dan-peso...,https://l.facebook.com/,...,True,False,True,mobile,Chrome,78,78.0.3904.62,Android,5.0.2,Mozilla/5.0 (Linux; Android 5.0.2; Mi 4i) Appl...


In [65]:
cols_to_group_by = ["canonical_url","type", "event_name", "status", "is_test", "device_type"]
# events_by_url = events.groupby(cols_to_group_by)
# events_by_url.head()

combined_raw_df = pd.DataFrame({"count":events.fillna(0).groupby(cols_to_group_by).size()}).reset_index()
combined_raw_df

,canonical_url,type,event_name,status,is_test,device_type,count
0,https://admin.moneysmart.id/,event,Reading,Article Body 100,False,desktop,6
1,https://admin.moneysmart.id/,event,Reading,Article Body 100,False,mobile,1
2,https://admin.moneysmart.id/,event,Reading,Article Body 25,False,desktop,7
3,https://admin.moneysmart.id/,event,Reading,Article Body 25,False,mobile,1
4,https://admin.moneysmart.id/,event,Reading,Article Body 50,False,desktop,7
5,https://admin.moneysmart.id/,event,Reading,Article Body 50,False,mobile,1
6,https://admin.moneysmart.id/,event,Reading,Article Body 75,False,desktop,7
7,https://admin.moneysmart.id/,event,Reading,Article Body 75,False,mobile,1
8,https://admin.moneysmart.id/,page,PageView,0,False,desktop,8
9,https://admin.moneysmart.id/,page,PageView,0,False,mobile,1


In [66]:

# join across both sides of the AB test
test_df =combined_raw_df[combined_raw_df.is_test==True]
test_df = test_df.rename(columns={"count":"test_count"}) #in_place = True
control_df = combined_raw_df[combined_raw_df.is_test==False]
control_df = control_df.rename(columns={"count":"control_count"})

print(test_df.head(20))

#print(control_df.head())



#print(test_df.keys)
test_df.reset_index()
#print(20*"-")
#print(test_df.keys())
#print(test_df.columns)
test_df
control_df.reset_index()
test_df = test_df.set_index(cols_to_group_by)
control_df = control_df.set_index(cols_to_group_by)

combined_df = test_df.join(control_df, rsuffix="_").reset_index()
#combined_df.fillna(0) #get rid of NaN

                                        canonical_url   type event_name  \
15                         https://www.moneysmart.id/   page   PageView   
16                         https://www.moneysmart.id/   page   PageView   
17                         https://www.moneysmart.id/   page   PageView   
31  https://www.moneysmart.id/10-aksi-kecil-tapi-r...   page   PageView   
32  https://www.moneysmart.id/10-alasan-kenapa-per...  event    Reading   
33  https://www.moneysmart.id/10-alasan-kenapa-per...  event    Reading   
34  https://www.moneysmart.id/10-alasan-kenapa-per...  event    Reading   
35  https://www.moneysmart.id/10-alasan-kenapa-per...  event    Reading   
37  https://www.moneysmart.id/10-alasan-kenapa-per...   page   PageView   
41  https://www.moneysmart.id/10-alasan-penolakan-...  event    Reading   
45  https://www.moneysmart.id/10-alasan-penolakan-...  event    Reading   
46  https://www.moneysmart.id/10-alasan-penolakan-...  event    Reading   
50  https://www.moneysmar

In [67]:
combined_df["test_fraction"] = combined_df.apply (lambda row: row.test_count/(row.control_count + row.test_count), axis=1)

In [83]:
events[events.canonical_url.str.endswith("https://www.moneysmart.id/")].groupby(["canonical_url"]).size()

canonical_url
https://www.moneysmart.id/    5352
dtype: int64

In [78]:
# combined_df[combined_df.test_fraction<0.05][(combined_df['event_name']=="PageView")  | (combined_df['status']=="Article Body 100")].sort_values(combined_df.test_fraction) #.to_csv("results.csv")
# combined_df.sort_values(combined_df.canonical_url).head()
combined_df.sort_values("canonical_url")

>> we're missing some events here e.g. if you look at homepage.  Likely need to fill with zeros or soemthign.

,canonical_url,type,event_name,status,is_test,device_type,test_count,control_count,test_fraction
0,https://www.moneysmart.id/,page,PageView,0,True,desktop,225,NaN,NaN
1,https://www.moneysmart.id/,page,PageView,0,True,mobile,337,NaN,NaN
2,https://www.moneysmart.id/,page,PageView,0,True,tablet,5,NaN,NaN
3,https://www.moneysmart.id/10-aksi-kecil-tapi-r...,page,PageView,0,True,mobile,1,NaN,NaN
4,https://www.moneysmart.id/10-alasan-kenapa-per...,event,Reading,Article Body 100,True,desktop,1,NaN,NaN
5,https://www.moneysmart.id/10-alasan-kenapa-per...,event,Reading,Article Body 25,True,desktop,1,NaN,NaN
6,https://www.moneysmart.id/10-alasan-kenapa-per...,event,Reading,Article Body 50,True,desktop,1,NaN,NaN
7,https://www.moneysmart.id/10-alasan-kenapa-per...,event,Reading,Article Body 75,True,desktop,1,NaN,NaN
8,https://www.moneysmart.id/10-alasan-kenapa-per...,page,PageView,0,True,desktop,1,NaN,NaN
15,https://www.moneysmart.id/10-alasan-penolakan-...,page,PageView,0,True,mobile,1,NaN,NaN


In [87]:
events[events.canonical_url == "https://www.moneysmart.id/"].groupby(["canonical_url", "is_test"]).size()

canonical_url               is_test
https://www.moneysmart.id/  False      4785
                            True        567
dtype: int64

# Error Logs


## Scalyr

https://www.scalyr.com/help/api#query

* 500s and then with sense check removing bots
* Where status != 200 or a 30x
* All with links to the query dashboard.

Do for:
* ID v1
* ID v2 front end


## Airbrake?? 

* Overall counts probably only

# Search Console.

* Position, clicks, impressions over time, with country filter set to ID
* Track top organic urls and see if there's a different trend

In [312]:
0.0584*24*30

42.048

# Digging Into What Makes Up a Split

* By URL any significant  discrepancy?
* Is it distorted by bots?

## Pages Performing Worse in v2 Than v1

Background to this is that we are seeing a drop off at the top of the page, and that might be due to specific pages, so we want to know if pages are performing poorly:

* Pageviews ratio
* Average scroll depth

# Query Params

In [ ]:
from urllib.parse import urlparse, parse_qs

def extract_query_params_from_url(url):
    url_components = urlparse(url)  # this is also a good way of getting the canonical.
    query_params_dict = parse_qs(url_components.query)
    return query_params_dict
    
    
def extract_reduced_query_params_from_url(url):
    """
    Excludes AMP, and groups together where theree's a unique value e.g. gclid, or fbclid
    """
    
    query_params = extract_query_params_from_url(url)
    unique_per_user_keys = ["fbclid", "gclid"]
    for key in unique_per_user_keys:
        if key in query_params:
            query_params = {key:["unique-per-use-other-keys-removed",]}  # doing this as facebook has loads of stuff
            #query_params[key]=['unqiue_per_user']
    #TODO: >> actually build this.
    #fbclid
    #gclid
    
    return query_params

def extract_reduced_query_params_from_url_to_sorted_tuple(url):
    """
    This is being done to make it hashable for grouping.
    """
    query_params_dict = extract_reduced_query_params_from_url(url)
    keys = list(query_params_dict.keys())
    keys.sort()
    ret = []
    for key in keys:
        ret.append((key, ",".join(query_params_dict[key])))
    
    return tuple(ret) 

non_amp_pageview_query_params = pageview_events[pageview_events.is_amp==False].apply(lambda row: extract_reduced_query_params_from_url_to_sorted_tuple(row["page_url"]), axis=1)
non_amp_pageview_query_params.head()

In [ ]:
non_amp_pageview_query_params[0]

In [ ]:
pqdf = pd.DataFrame()
pqdf["query_params"]=non_amp_pageview_query_params
pqdf["sent_at_timestamp"] = pageview_events[pageview_events.is_amp==False]["sent_at_timestamp"]

In [ ]:
pqdf.head()

In [ ]:
pqdf.dtypes

In [ ]:
query_params_by_date = pqdf.groupby([pqdf.sent_at_timestamp.dt.floor('d'), pqdf.query_params]).count()

In [ ]:
query_params_by_date #.sort_values(query_params_by_date.sent_at_timestamp)